# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 6 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [2]:
train = pd.read_csv('../../preprocessing/train.csv')
test = pd.read_csv('../../preprocessing/test.csv')

In [3]:
train.head()

,date,title,description,category,text,source,frequent_category
0,2023-05-04 17:18:27,Fans bemerken Fehler sofort - FDP-Minister fei...,Justizminister Marco Buschmann hat mit einem V...,Panorama,NaN,Focus,Other
1,2022-10-24 12:00:00,"""Rishi Sunak ist auf der Siegerstraße""",Auf der Suche nach einer Nachfolge für das Amt...,ZDF-Korrespondent Stamm,NaN,ZDF heute,Other
2,2023-04-18 19:00:00,Nach Springer-Enthüllungen: Vom Clickbait-Boul...,In einem Interview verteidigte Autorin Nora Bo...,Missing,NaN,taz,Missing
3,2023-04-25 16:44:58,Jetzt droht der Ausverkauf der nächsten deutsc...,"Erfolgreich, traditionell, familienorientiert ...",Energie,NaN,Welt,Other
4,2023-06-12 10:08:32,UBS schließt Übernahme der Credit Suisse ab,Die Übernahme der Großbank Credit Suisse ist n...,Missing,NaN,Tagesschau,Missing


In [4]:
test.head()

,date,title,description,category,text,source,frequent_category
0,2023-03-14 07:56:23,Bundesweite Warnstreiks im Gesundheitswesen,Die Gewerkschaft ver.di hat im Tarifstreit des...,Missing,NaN,Tagesschau,Missing
1,2022-11-22 07:20:06,Gräuel im Ukraine-Krieg - Putins Männer holen ...,Die russische Armee hat in der Ukraine einen K...,Ukraine-Krise,NaN,Focus,Ukraine-Krise
2,2023-05-10 08:45:00,Der Speckgürtel um Metropolen wächst,"Wohnen wird in Metropolen immer teurer, deshal...",So wohnen die Deutschen,NaN,ZDF heute,Other
3,2022-12-31 04:28:49,Nordkorea feuert drei Kurzstreckenraketen ab,Kurz vor Jahresende feuert Nordkorea nach Anga...,Missing,NaN,Tagesschau,Missing
4,2023-08-15 10:20:54,Ermittlungen: Schüsse auf Shisha-Bar in Altona...,NaN,News,NaN,Zeit,News


## Load German stopwords

In [5]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [6]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title' 

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [9]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [10]:
dtm

<54001x21678 sparse matrix of type '<class 'numpy.int64'>'
	with 277324 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.decomposition import LatentDirichletAllocation

In [12]:
lda = LatentDirichletAllocation(n_components=6, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=6, random_state=42)

## Analyse extracted features

In [13]:
len(cv.get_feature_names_out())

21678

In [14]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'übung', 'übungen', 'украинцы'],
      dtype=object)

In [15]:
len(lda.components_)

6

In [16]:
lda.components_

array([[  0.166668  , 161.10814132,   0.16666738, ...,   7.13780647,
          4.16544739,   0.16667269],
       [  0.16786866,  17.09637577,   2.1662808 , ...,   0.16822314,
          0.16666935,   0.16667362],
       [  0.16666783,  26.36163626,   0.1670495 , ...,   0.16697355,
          0.16923635,   2.16663446],
       [  2.16301306,  51.75844945,   0.1666674 , ...,   0.16791297,
          1.16510701,   0.16667252],
       [  0.16772587,  87.585876  ,   0.16666742, ...,   0.16710942,
          0.16687068,   0.1666731 ],
       [  0.16805658,   8.08952121,   0.1666675 , ...,   2.19197444,
          0.16666923,   0.16667362]])

In [17]:
len(lda.components_[0])

21678

### Show most important words of first extracted topic

In [18]:
first_topic = lda.components_[0]

In [19]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 1250,  2335, 14121, ..., 15369,  5574, 18605], dtype=int64)

In [20]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [21]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

usa
deutschland
nato
marktbericht
scholz
china
krieg
russland
eu
ukraine


### Show most important words of first extracted topic

In [22]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['neue', 'putin', 'analyse', 'polen', 'baerbock', 'usa', 'deutschland', 'nato', 'marktbericht', 'scholz', 'china', 'krieg', 'russland', 'eu', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['prozess', 'pläne', 'israel', 'kinder', 'ende', 'deutschland', 'stream', 'usa', 'mann', 'wegen', 'nord', 'menschen', 'habeck', 'trump', 'frauen']


Die TOP-15 Wörter für das Thema #2
['kiew', 'putins', 'pandemie', 'invasion', 'scholz', 'china', 'un', 'us', 'lage', 'russland', 'russische', 'putin', 'corona', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #3
['grünen', 'union', 'bundesliga', 'fdp', 'fordert', 'wegen', 'türkei', 'wahl', 'polizei', 'ampel', 'afd', 'spd', 'deutschland', 'cdu', 'berlin']


Die TOP-15 Wörter für das Thema #4
['israel', 'ticket', 'inflation', 'geht', 'ex', 'regierung', 'italien', 'wm', 'frankreich', 'bayern', 'präsident', 'tote', 'deutsche', 'euro', 'us']


Die TOP-15 Wörter für das Thema #5
['jahre', 'haft', 'jahren', 'frau', 'zei

### Get topics with highest probability for news items in train data

In [23]:
topic_results = lda.transform(dtm)
topic_results.shape

(54001, 6)

In [24]:
topic_results[0].round(5)

array([0.01678, 0.01679, 0.01669, 0.01692, 0.91604, 0.01677])

In [25]:
topic_results[0].argmax()

4

In [26]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [27]:
train.head()

,date,title,description,category,text,source,frequent_category,topic
0,2023-05-04 17:18:27,Fans bemerken Fehler sofort - FDP-Minister fei...,Justizminister Marco Buschmann hat mit einem V...,Panorama,NaN,Focus,Other,4
1,2022-10-24 12:00:00,"""Rishi Sunak ist auf der Siegerstraße""",Auf der Suche nach einer Nachfolge für das Amt...,ZDF-Korrespondent Stamm,NaN,ZDF heute,Other,4
2,2023-04-18 19:00:00,Nach Springer-Enthüllungen: Vom Clickbait-Boul...,In einem Interview verteidigte Autorin Nora Bo...,Missing,NaN,taz,Missing,5
3,2023-04-25 16:44:58,Jetzt droht der Ausverkauf der nächsten deutsc...,"Erfolgreich, traditionell, familienorientiert ...",Energie,NaN,Welt,Other,0
4,2023-06-12 10:08:32,UBS schließt Übernahme der Credit Suisse ab,Die Übernahme der Großbank Credit Suisse ist n...,Missing,NaN,Tagesschau,Missing,1


In [28]:
#Save result to csv
train.to_csv('train_lda_topics_6.csv')